In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ctypes
# Assuming the Python code above is saved as 'vertex_functions_no_numpy.py'
import os
from sl2cfoam import *
import numpy as np
from prettytable import PrettyTable
import json

In [3]:
clib = ctypes.CDLL("libsl2cfoam.so")

In [4]:
js=[1, 1, 1, 1, 2, 1, 1, 1, 1]
is_intertwiners=[1, 1, 1, 1, 1,]
Dl=1

In [5]:
js_c = (ctypes.c_int * len(js))(*[ctwo(j) for j in js])
is_c = (ctypes.c_int * len(is_intertwiners))(*[ctwo(i) for i in is_intertwiners])

In [6]:
sl2c_data_folder = "/home/akogios/Projects/sl2cfoam-next/data_sl2cfoam"
#sl2c_config = Config(Verbosity.VerbosityOff, Accuracy.VeryHighAccuracy, 100, 0)
sl2c_config = Config(Verbosity.VerbosityOff, Accuracy.VeryHighAccuracy, 100, 0)
Immirzi = 1.0

In [7]:
j=10;

In [8]:

sl2c_config.verbosity = Verbosity.VerbosityOff
sl2c_config.accuracy = Accuracy.VeryHighAccuracy
sl2c_config.max_spin = 100
sl2c_config.max_MB_mem_per_thread = 0
Immirzi = 1.0 

In [9]:
cinit(sl2c_data_folder, Immirzi , sl2c_config)

In [10]:
boundary_data_dict = {
    "00": {
        "tetra2345": [(1.20253, 2.82107), (2.21430, -2.48253), (1.02395, -0.47668), (1.85459, 0.65906)],
        "tetra1345": [(2.21430, -0.65906), (1.02395, -2.66491), (1.85459, 2.48253), (1.20253, 0.32052)],
        "tetra1245": [(1.77215, -1.57080), (3.14159, 0), (0.92730, 0.65906), (0.92730, 2.48253)],
        "tetra2356": [(1.02395, -0.47668), (1.85459, 0.65906), (1.20253, 2.82107), (2.21430, -2.48253)],
        "tetra1356": [(2.21430, -0.65906), (1.20253, 0.32052), (1.02395, -2.66491), (1.85459, 2.48253)],
        "tetra1256": [(0.92730, 0.65906), (1.77215, -1.57080), (3.14159, 0), (0.92730, 2.48253)],
        "tetra2346": [(1.20253, 2.82107), (1.85459, 0.65906), (1.02395, -0.47668), (2.21430, -2.48253)],
        "tetra1346": [(1.85459, 2.48253), (1.02395, -2.66491), (2.21430, -0.65906), (1.20253, 0.32052)],
        "tetra1246": [(0.92730, 0.65906), (0.92730, 2.48253), (3.14159, 0), (1.77215, -1.57080)],
    }
}

In [11]:
delta = "00"  # Select desired delta
tetra_data = boundary_data_dict[delta];

In [12]:
j123 = j;
j124 = j;
j125 = j;
j134 = j;
j135 = j;
j145 = j;
j234 = j;
j235 = j;
j245 = j;
j345 = j;

In [13]:
# Extract theta and phi values for each boundary tetrahedron
# First 4-simplex
thetas_1245 = [theta for (theta, phi) in tetra_data["tetra1245"]];
phis_1245 = [phi for (theta, phi) in tetra_data["tetra1245"]];
thetas_1345 = [theta for (theta, phi) in tetra_data["tetra1345"]];
phis_1345 = [phi for (theta, phi) in tetra_data["tetra1345"]];
thetas_2345 = [theta for (theta, phi) in tetra_data["tetra2345"]];
phis_2345 = [phi for (theta, phi) in tetra_data["tetra2345"]];

In [14]:
cs1245 = coherentstate_compute([j125, j124, j245, j145], [list(element) for element in tetra_data["tetra1245"]])
cs1345 = coherentstate_compute([j145, j135, j134, j345], [list(element) for element in tetra_data["tetra1345"]])
cs2345 = coherentstate_compute([j345, j245, j235, j234], [list(element) for element in tetra_data["tetra2345"]])

In [15]:
vertex = vertex_BF_compute([j, j, j, j, j, j, j, j, j, j])

In [16]:
contracted_vertex = np.dot(np.array(vertex.a), np.array(cs1245.a))

In [17]:
# 1. Define the path to the .npy file exported by Julia
file_path = "julia_array.npy"

print("--- Python Import from .npy ---")
try:
    # 2. Load the .npy file directly into a NumPy array
    python_numpy_array = np.load(file_path)

    # 3. Print details and a slice to verify
    print(f"NumPy array successfully loaded from '{file_path}':")
    print(f"Shape: {python_numpy_array.shape}")
    print(f"Data type: {python_numpy_array.dtype}")
    print("First few elements (top-left corner):")
    print(python_numpy_array[:3,:3])

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please ensure you ran the Julia script ('export_to_npy.jl') first in the same directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

--- Python Import from .npy ---
NumPy array successfully loaded from 'julia_array.npy':
Shape: (21, 21, 21, 21)
Data type: complex128
First few elements (top-left corner):
[[[[ 1.98343816e-08-5.58630714e-08j -6.17455844e-08-9.62107684e-08j
    -2.39757995e-07+2.71112493e-08j ...  1.81835133e-09-4.28533319e-09j
    -3.66728585e-10-4.86448318e-10j -3.85989504e-11+4.75067444e-12j]
   [-3.43541566e-08+9.67576779e-08j  1.05974249e-07+1.65127012e-07j
     4.03947400e-07-4.56773868e-08j ...  1.74652809e-09-4.11606640e-09j
    -4.61958212e-10-6.12765965e-10j -6.07775847e-11+7.48037227e-12j]
   [ 4.43510255e-08-1.24913625e-07j -1.34301852e-07-2.09266532e-07j
    -4.92652706e-07+5.57079665e-08j ... -1.43031695e-10+3.37084732e-10j
    -2.46333136e-10-3.26749385e-10j -6.48176941e-11+7.97762011e-12j]
   ...
   [ 1.20647833e-07-3.39801797e-07j  2.08278249e-07+3.24535115e-07j
     5.13030807e-08-5.80122724e-09j ...  1.78951283e-14-4.21736911e-14j
     2.25650949e-16+2.99315431e-16j -7.15368500e-19+8.

In [ ]:
(contracted_vertex  - python_numpy_array)/python_numpy_array *1

array([[[[ 9.05156059e-16+3.88389106e-15j,
           0.00000000e+00-0.00000000e+00j,
          -1.91822902e-16-2.56295271e-16j, ...,
          -4.87977327e-16+4.30886034e-15j,
          -3.44351664e-15-7.89306016e-16j,
           5.70132222e-15-5.51386920e-15j],
         [ 8.22997768e-16+3.91711083e-16j,
          -7.03470223e-17-1.88859129e-15j,
           4.10612581e-16-1.31327053e-15j, ...,
          -1.43974320e-15+1.81669460e-15j,
          -9.81397384e-16-6.30825605e-15j,
          -2.13993284e-15-1.19860212e-14j],
         [ 2.65125007e-15-8.88359570e-16j,
           1.24755542e-15+4.21183756e-16j,
          -6.74240739e-18-1.84098361e-15j, ...,
          -1.17867946e-16+4.42106739e-15j,
          -8.11125060e-15+5.56479881e-17j,
           1.61675135e-15-2.28567260e-14j],
         ...,
         [-1.46872480e-15-1.50386434e-15j,
           1.02689470e-15-7.50248111e-17j,
          -9.43716782e-16+3.60865702e-16j, ...,
          -2.19807326e-15+1.09663702e-16j,
          -8.6281

: 

In [ ]:
#_mock_cinit_for_demo()
print("Library state mocked as initialized.")

# --- Example 1: Compute a single vertex amplitude ---
print("\n--- Computing Vertex Amplitude ---")
js_amp = [0.5, 1, 1.5, 2, 0.5, 1, 1.5, 2, 0.5, 1]
is_amp = [0.5, 1, 1.5, 2, 2.5]
dl_amp = 10
try:
    amplitude = vertex_amplitude(js_amp, is_amp, dl_amp)
    print(f"Computed Vertex Amplitude: {amplitude}")
except Exception as e:
    print(f"Error computing amplitude: {e}")

# --- Example 2: Compute a full vertex tensor (full range) ---
print("\n--- Computing Full Vertex Tensor ---")
js_full = [0.5, 1, 1.5, 2, 0.5, 1, 1.5, 2, 0.5, 1]
dl_full = 5
try:
    vertex_full = vertex_compute(js_full, dl_full)
    if vertex_full:
        print(f"Full Vertex Tensor computed. Shape: {vertex_full.shape}")
        # print(vertex_full.a) # Uncomment to view the tensor data (will be a nested list)
        print(f"First element: {vertex_full.a}") # Access first element or slice
    else:
        print("Full Vertex Tensor computation returned None.")
except Exception as e:
    print(f"Error computing full vertex tensor: {e}")

# --- Example 3: Compute a vertex tensor with specific intertwiner ranges ---
print("\n--- Computing Ranged Vertex Tensor ---")
js_range = [0.5, 1, 1.5, 2, 0.5, 1, 1.5, 2, 0.5, 1]
dl_range = 5
irange_compute = [
    (0.5, 1.5), (0.5, 1.5), (0.5, 1.5), (0.5, 1.5), (0.5, 1.5)
]
try:
    vertex_ranged = vertex_compute(js_range, dl_range, irange=irange_compute)
    if vertex_ranged:
        print(f"Ranged Vertex Tensor computed. Shape: {vertex_ranged.shape}")
    else:
        print("Ranged Vertex Tensor computation returned None.")
except Exception as e:
    print(f"Error computing ranged vertex tensor: {e}")

# --- Example 4: Load a vertex tensor (by spins) ---
print("\n--- Loading Vertex Tensor by Spins ---")
try:
    loaded_vertex_by_spins = vertex_load(js_full, dl_full)
    if loaded_vertex_by_spins:
        print(f"Vertex Tensor loaded by spins. Shape: {loaded_vertex_by_spins.shape}")
    else:
        print("Vertex Tensor loading by spins returned None.")
except Exception as e:
    print(f"Error loading vertex tensor by spins (expected if not saved): {e}")


# --- Example 5: Load a vertex tensor (by path) ---
print("\n--- Loading Vertex Tensor by Path ---")
# IMPORTANT: Change this to an actual path to a file saved by your C library
dummy_file_path = "/path/to/your/saved_vertex_tensor.bin"
try:
    if os.path.exists(dummy_file_path):
        loaded_vertex_by_path = vertex_load(dummy_file_path)
        if loaded_vertex_by_path:
            print(f"Vertex Tensor loaded by path. Shape: {loaded_vertex_by_path.shape}")
        else:
            print("Vertex Tensor loading by path returned None.")
    else:
        print(f"File not found at {dummy_file_path}. Skipping path load test.")
except Exception as e:
    print(f"Error loading vertex tensor by path: {e}")

Library state mocked as initialized.

--- Computing Vertex Amplitude ---
Computed Vertex Amplitude: 0.0

--- Computing Full Vertex Tensor ---


In [ ]:
        print(f"Ranged Vertex Tensor computed. Shape: {vertex_ranged.shape}")
    else:
        print("Ranged Vertex Tensor computation returned None.")
except Exception as e:
    print(f"Error computing ranged vertex tensor: {e}")

# --- Example 4: Load a vertex tensor (by spins) ---
print("\n--- Loading Vertex Tensor by Spins ---")
try:
    loaded_vertex_by_spins = vertex_load(js_full, dl_full)
    if loaded_vertex_by_spins:
        print(f"Vertex Tensor loaded by spins. Shape: {loaded_vertex_by_spins.shape}")
    else:
        print("Vertex Tensor loading by spins returned None.")
except Exception as e:
    print(f"Error loading vertex tensor by spins (expected if not saved): {e}")


# --- Example 5: Load a vertex tensor (by path) ---
print("\n--- Loading Vertex Tensor by Path ---")
# IMPORTANT: Change this to an actual path to a file saved by your C library
dummy_file_path = "/path/to/your/saved_vertex_tensor.bin"
try:
    if os.path.exists(dummy_file_path):
        loaded_vertex_by_path = vertex_load(dummy_file_path)
        if loaded_vertex_by_path:
            print(f"Vertex Tensor loaded by path. Shape: {loaded_vertex_by_path.shape}")
        else:
            print("Vertex Tensor loading by path returned None.")
    else:
        print(f"File not found at {dummy_file_path}. Skipping path load test.")
except Exception as e:
    print(f"Error loading vertex tensor by path: {e}")